# Webscraping

Im foglenden Notebook wird die Website https://issuu.com/fhnw/docs/modultabelle_20maschinenbau gescrpat, damit wir die Modulinformationen zu dem Studiengang Maschinenbau erhalten.

In [7]:
import pdfplumber
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [8]:
driver = webdriver.Chrome()  # oder z.B. webdriver.Firefox()
driver.get("https://issuu.com/fhnw/docs/modultabelle_20maschinenbau")

soup = BeautifulSoup(driver.page_source, "html.parser")
iframe = soup.find("iframe")
if iframe:
    src_url = iframe["src"]
    print("PDF/View URL:", src_url)
else:
    print("Kein iframe gefunden!")

driver.quit()

PDF/View URL: https://issuu.com/rd4?p=1&d=modultabelle_20maschinenbau&u=fhnw


In [9]:
pdf_path = "../data/Modultabelle Maschinenbau_HS2025_updated.pdf"
data_rows = []

# Hilfsfunktion zum Erkennen von Modulgruppen
def is_modulgruppe(text):
    if not text:
        return False
    # enthält ":" und kein Wort "Minimum" oder "alle"
    text = text.lower()
    return (":" in text) and ("minimum" not in text) and ("alle" not in text)

# Hilfsfunktion zum Erkennen gültiger Moduldatenzeilen
def is_gültige_modulzeile(row):
    if not row or len(row) < 2:
        return False
    # Modulname mindestens vorhanden und kein einleitender Text wie "Minimum" etc.
    modul = row[0]
    if modul is None:
        return False
    modul_lower = modul.lower()
    if "minimum" in modul_lower or "alle" in modul_lower:
        return False
    if modul.strip() == "":
        return False
    return True

with pdfplumber.open(pdf_path) as pdf:
    current_modulgruppe = ""
    for page in pdf.pages:
        tables = page.extract_tables()
        for table in tables:
            for row in table:
                # Prüfen ob Modulgruppe
                if row[0] and is_modulgruppe(row[0]):
                    current_modulgruppe = row[0].strip()
                    continue
                # Überspringe Header oder unerwünschte Zeilen
                if not is_gültige_modulzeile(row):
                    continue
                # Extrahiere Moduleinträge
                modul = row[0] if row[0] else ""
                Kürzel = row[1] if len(row) > 1 and row[1] else ""
                voraussetzung = row[4] if len(row) > 4 and row[4] else ""
                
                # Semesterfindung (Spalten nach Nr. 4 durchsuchen)
                semester = ""
                for i in range(5, len(row)):
                    if row[i] and row[i].strip():
                        semester = str(i - 4)
                        break

                data_rows.append({
                    "Modulgruppe": current_modulgruppe,
                    "Modul": modul.strip(),
                    "Kürzel": Kürzel.strip(),
                    "Voraussetzung": voraussetzung.strip(),
                    "Semester": semester
                })

df = pd.DataFrame(data_rows)

In [10]:
df

,Modulgruppe,Modul,Kürzel,Voraussetzung,Semester
0,Grundlagen: Mathematik 1,Lineare Algebra 1,lalg1,,11
1,Grundlagen: Mathematik 1,Lineare Algebra 2,lalg2,,4
2,Grundlagen: Mathematik 1,Informatik (M),infM,,4
3,Grundlagen: Mathematik 1,Wahrscheinlichkeitstheorie und Statistik,wst,,4
4,Grundlagen: Mathematik 1,Datenanalyse (Machine Learning),dan,,4
...,...,...,...,...,...
87,Projekte: Maschinenbau,Kontext,min. Anzahl Module,,
88,Projekte: Maschinenbau,Kommunikation,2,,
89,Projekte: Maschinenbau,Englisch,4,,
90,Projekte: Maschinenbau,Betriebswirtschaftslehre,2,,


In [11]:
data = pd.read_excel('../data/_data.xlsx')
data.to_csv('../data/data.csv')

## Data Wrangling

In [59]:
from itertools import combinations


data = pd.read_csv('../data/data.csv', sep=',', dtype=str)
data = data.drop('Unnamed: 0', axis=1, errors='ignore')
data = data.rename(columns={'Kürzel': 'Kuerzel'})


# Duplikate checken + bereinigen
print("Duplikate Kuerzel:", data['Kuerzel'].duplicated().sum())
data = data.drop_duplicates(subset=['Kuerzel'], keep='first')


# 1. Nodes: unique Kuerzel
kuerzel_list = data['Kuerzel'].dropna().str.strip().unique().tolist()
nodes_data = []
for kuerzel in kuerzel_list:
    row = data[data['Kuerzel'] == kuerzel].iloc[0]  # erste Zeile nehmen
    nodes_data.append({
        'Id': kuerzel,
        'Label': kuerzel,
        'Name': str(row['Modul']),
        'Gruppe': str(row['Modulgruppe']),
        'Semester': str(row['Semester'])
    })


nodes_df = pd.DataFrame(nodes_data)
nodes_df.to_csv('../data/nodes.csv', index=False)
print(f"Nodes: {len(nodes_df)}")


kuerzel_set = set(kuerzel_list)


# 2. DREI separate Edge-Dateien
edges_pre = []
missing_sources = set()  # Sammle fehlende Source-Module
for _, row in data.iterrows():
    target = str(row['Kuerzel']).strip()
    voraus_raw = str(row['Voraussetzung']).strip()
    if voraus_raw not in ['nan', 'None', '']:
        for source in [p.strip() for p in voraus_raw.split(',') if p.strip()]:
            if source in kuerzel_set:
                edges_pre.append({'Source': source, 'Target': target, 'Type': 'Directed', 'Label': 'Voraussetzung'})
            else:
                missing_sources.add((source, target))
if missing_sources:
    print(f"Hinweis: {len(missing_sources)} Voraussetzungen mit Source-Modulen, die nicht im Curriculum existieren (korrekt ignoriert):")
    for source, target in sorted(missing_sources):
        print(f"  - '{source}' → '{target}' (Modul '{source}' ist kein Modul im Curriculum)")
pd.DataFrame(edges_pre).to_csv('../data/edges_pre_req.csv', index=False)
print(f"Edges Pre-Req: {len(edges_pre)}")


# Gleiche Gruppe - ANGEPASST: Label enthält jetzt die Modulgruppe
group_dict = data.groupby('Modulgruppe')['Kuerzel'].apply(lambda x: x.dropna().str.strip().tolist()).to_dict()
edges_group = []
for gruppe, mods in group_dict.items():
    if len(mods) >= 2:
        for mod1, mod2 in combinations(sorted(set(mods)), 2):
            edges_group.append({
                'Source': mod1, 
                'Target': mod2, 
                'Type': 'Undirected', 
                'Label': str(gruppe).strip()  # Modulgruppe als Label
            })
pd.DataFrame(edges_group).to_csv('../data/edges_same_group.csv', index=False)
print(f"Edges Same Group: {len(edges_group)}")


# Gleiches Semester - KORRIGIERT: Module mit mehreren Semestern werden mit Modulen in ALLEN Semestern verbunden
edges_sem = []
for _, row1 in nodes_df.iterrows():
    modul1 = row1['Id']
    semester1 = str(row1['Semester']).strip()
    
    if pd.notna(semester1) and semester1 not in ['nan', 'None', '']:
        # Prüfe, ob mehrere Semester durch Semikolon getrennt sind
        if ';' in semester1:
            semester_list = [s.strip() for s in semester1.split(';')]
        else:
            semester_list = [semester1]
        
        # Für jedes Semester Verbindungen erstellen
        for sem in semester_list:
            # Finde alle Module desselben Semesters
            # Prüfe sowohl auf exakte Übereinstimmung als auch auf Semikolon-getrennte Semester
            try:
                sem_float = float(sem)
                semester_str = nodes_df['Semester'].astype(str)
                same_semester = nodes_df[
                    ((semester_str.str.contains(f'^{sem}(;|$)', regex=True, na=False)) |
                     (nodes_df['Semester'] == sem_float)) &
                    (nodes_df['Id'] != modul1)
                ]
            except (ValueError, TypeError):
                # Falls Konvertierung fehlschlägt, verwende nur String-Match
                semester_str = nodes_df['Semester'].astype(str)
                same_semester = nodes_df[
                    (semester_str.str.contains(f'^{sem}(;|$)', regex=True, na=False)) &
                    (nodes_df['Id'] != modul1)
                ]
            
            for _, row2 in same_semester.iterrows():
                modul2 = row2['Id']
                # Vermeide Duplikate (ungerichtete Kanten)
                if modul1 < modul2:
                    label_sem = sem.split('.')[0] if '.' in sem else sem  # Entferne .0 bei Float-Strings
                    edges_sem.append({
                        'Source': modul1,
                        'Target': modul2,
                        'Type': 'Undirected',
                        'Label': f'Semester {label_sem}'
                    })


# Entferne Duplikate (könnte vorkommen wenn ein Modul mehrfach mit demselben Semester verbunden wird)
edges_sem_df = pd.DataFrame(edges_sem)
edges_sem_df = edges_sem_df.drop_duplicates(subset=['Source', 'Target'])
edges_sem_df.to_csv('../data/edges_same_semester.csv', index=False)
print(f"Edges Same Semester: {len(edges_sem_df)}")


Duplikate Kuerzel: 1
Nodes: 87
Hinweis: 2 Voraussetzungen mit Source-Modulen, die nicht im Curriculum existieren (korrekt ignoriert):
  - 'bb' → 'wch' (Modul 'bb' ist kein Modul im Curriculum)
  - 'ten1' → 'pro4M' (Modul 'ten1' ist kein Modul im Curriculum)
Edges Pre-Req: 10
Edges Same Group: 539
Edges Same Semester: 768


C:\Users\Sandra\AppData\Local\Temp\ipykernel_24876\2481399478.py:93: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((semester_str.str.contains(f'^{sem}(;|$)', regex=True, na=False)) |
C:\Users\Sandra\AppData\Local\Temp\ipykernel_24876\2481399478.py:93: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((semester_str.str.contains(f'^{sem}(;|$)', regex=True, na=False)) |
C:\Users\Sandra\AppData\Local\Temp\ipykernel_24876\2481399478.py:93: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((semester_str.str.contains(f'^{sem}(;|$)', regex=True, na=False)) |
C:\Users\Sandra\AppData\Local\Temp\ipykernel_24876\2481399478.py:93: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the grou